In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper
from series_extraction.excel_validator import ExcelValidator

from ast_transformation.series_formula_generator import SeriesFormulaGenerator
from ast_transformation.formula_generator import SeriesIdLoader

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from pipeline_building.series_dependencies_builder import SeriesDependenciesBuilder

from excel_builder import ExcelBuilder

from excel_checker import ExcelChecker


In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_1'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")
excel_reduced_clean_filepath = os.path.join(data_directory, "excel_files_reduced_clean", f"{project_name}_reduced_clean.xlsx")
excel_reduced_clean_series_filepath = os.path.join(data_directory, "excel_files_reduced_clean_series", f"{project_name}_reduced_clean_series.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)
is_valid = ExcelValidator.validate_excel(excel_reduced)
if not is_valid:
    raise Exception("Excel file is not valid")

excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)
ExcelBuilder.create_excel_from_workbook(excel_reduced_clean.workbook_with_formulas, excel_reduced_clean_filepath)

extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)
if not is_compatible:
    print(extracted_tables)
    raise Exception("Excel file is not compatible")

series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)
series_mapping = SeriesMapper.map_series(series_dict)
series_iterator = SeriesIterator.iterate_series(series_dict)

series_list = [series for series in series_iterator]

series_list_with_formulas = [
    series for series in series_list if series.formulas != [None, None]
]
series_list_with_values = [
    series for series in series_list if series.formulas == [None, None]
]

series_list_new = []
formula_1_ast_series_list = []

for series in series_list_with_formulas:
    formula_1, formula_2 = SeriesFormulaGenerator.adjust_formulas(
        series.formulas
    )
    if formula_1 is not None and formula_2 is not None:

        series_implementer = SeriesImplementer(
            series_mapping, sheet_name=series.worksheet.sheet_name
        )

        formula_1_ast = FormulaParser.parse_formula(formula_1)
        formula_1_ast_series = series_implementer.update_ast(formula_1_ast)
        formula_1_ast_series_list.append((series.series_id, formula_1_ast_series))

        formula_2_ast = FormulaParser.parse_formula(formula_2)
        formula_2_ast_series = series_implementer.update_ast(formula_2_ast)

        SeriesFormulaGenerator.process_series_formulas(
            series,
            formula_1_ast_series,
            formula_2_ast_series,
            series_mapping,
            series_dict,
            series_list_new,
        )

series_list_updated = series_list_new + series_list_with_values

ExcelBuilder.create_excel_from_series(series_list_updated, excel_reduced_clean_series_filepath)
ExcelChecker.excels_are_equivalent(excel_reduced_clean_filepath, excel_reduced_clean_series_filepath)

True

In [4]:
# Assuming formula_1_ast_series_list is a list of tuples, each containing a series_id and its corresponding AST

# Initialize the adjacency list dictionary
dag_adjacency_list = {}

# Iterate through each series_id and its corresponding formula AST
for series_id, formula_1_ast_series in formula_1_ast_series_list:
    # Get the list of dependent series_ids from the AST
    dependencies_series_ids = SeriesDependenciesBuilder.get_series_ids(formula_1_ast_series)
    
    # Check if the series_id is already in the DAG adjacency list
    if series_id not in dag_adjacency_list:
        dag_adjacency_list[series_id] = []
    
    # Add the dependencies to the series_id's list in the DAG
    dag_adjacency_list[series_id].extend(dependencies_series_ids)

# Optional: Remove duplicates if necessary (depends on the application's needs)
for key, values in dag_adjacency_list.items():
    # Convert the list to a set and back to a list to remove duplicates
    dag_adjacency_list[key] = list(set(values))

# Printing the DAG adjacency list to see the structure
print(dag_adjacency_list)


{SeriesId(sheet_name='Sheet1', series_header='horizontal_column_1', series_header_cell_row=12, series_header_cell_column=2): [SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)], SeriesId(sheet_name='Sheet1', series_header='horizontal_column_2', series_header_cell_row=13, series_header_cell_column=2): [SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)], SeriesId(sheet_name='Sheet1', series_header='col_3', series_header_cell_row=2, series_header_cell_column=4): [SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3), SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)], SeriesId(sheet_name='Sheet1', series_header='col_4', series_header_cell_row=2, series_header_cell_column=5): [SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_head

In [5]:
dag_adjacency_list

{SeriesId(sheet_name='Sheet1', series_header='horizontal_column_1', series_header_cell_row=12, series_header_cell_column=2): [SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)],
 SeriesId(sheet_name='Sheet1', series_header='horizontal_column_2', series_header_cell_row=13, series_header_cell_column=2): [SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)],
 SeriesId(sheet_name='Sheet1', series_header='col_3', series_header_cell_row=2, series_header_cell_column=4): [SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3),
  SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2)],
 SeriesId(sheet_name='Sheet1', series_header='col_4', series_header_cell_row=2, series_header_cell_column=5): [SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series